In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import numpy as np
from google.cloud import vision
import io
from jarowinkler import jarowinkler_similarity
import requests
from requests.adapters import HTTPAdapter
from urllib3.util import Retry

In [3]:
url = "https://appfarmacitymicroservice-prod.azurewebsites.net/api/Medicine/search?filter=azitromicina"
#for word in predicted_words:
#    url = url + word + "%20"
#print(url)

#print('before request')
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)
response = session.get(url)

print(response.json())
#pd.read_json(response.json(), orient ='id')

[{'package': {'id': 34395, 'description': None, 'potency': '500', 'unitPotencyId': 3, 'unitPotency': {'id': 3, 'description': 'mg'}, 'packageDescription': {'id': 2411, 'description': 'Comprimidos x 3'}}, 'formula': {'id': 244, 'description': 'azitromicina                    '}, 'pharmacologicalAction': {'id': 4, 'description': 'Antibiótico'}, 'saleType': {'id': 3, 'description': 'Venta Bajo Receta Archivada'}, 'id': 1071, 'cuf': 107585, 'barCode': '7798011651559', 'description': 'AZITRAL  500 MG COMP.X 3', 'indications': 'Tipo de Venta: Venta Bajo Receta Archivada. ', 'considerations': ' ', 'adverseEffects': '', 'posology': '', 'margin': 74.82314049586776, 'publicPrice': 667.92, 'allowsReservation': False, 'medicalLaboratory': {'id': 174, 'abbreviation': 'LEPETIT'}}, {'package': {'id': 36595, 'description': None, 'potency': '500', 'unitPotencyId': 3, 'unitPotency': {'id': 3, 'description': 'mg'}, 'packageDescription': {'id': 2411, 'description': 'Comprimidos x 3'}}, 'formula': {'id': 2

In [5]:
full_dict = response.json()[0]

In [6]:
full_dict.keys()

dict_keys(['package', 'formula', 'pharmacologicalAction', 'saleType', 'id', 'cuf', 'barCode', 'description', 'indications', 'considerations', 'adverseEffects', 'posology', 'margin', 'publicPrice', 'allowsReservation', 'medicalLaboratory'])

In [13]:

prices= []
ids = []
description = []
labs = []

for drug in response.json():
    prices.append(drug['publicPrice'])
    ids.append(drug['id'])
    description.append(drug['description'])
    labs.append(drug['medicalLaboratory']['abbreviation'])

df = pd.DataFrame(list(zip(ids, description, labs, prices)), columns =['id', 'description', 'lab', 'prices']) 


In [14]:
df

,id,description,lab,prices
0,1071,AZITRAL 500 MG COMP.X 3,LEPETIT,667.92
1,1076,AZITROX 500 MG COMP.X 3,AUSTRAL,754.16
2,6405,OROBIOTIC 500 MG COMP.X 9,FORTBENTON,838.65
3,1077,AZITROX 200 MG/5 ML SUSP.X 15 ML,AUSTRAL,860.00
4,1825,CETAXIM 500 MG COMP.X 3,TRB PHARMA,880.00
5,6406,OROBIOTIC 500 MG COMP.X 12,FORTBENTON,890.36
6,1935,CLEARSING 500 MG COMP.X 3,DUNCAN,895.69
7,5742,NEBLIC 500 MG COMP.X 3,LAZAR,900.99
8,8950,ZITROMAX 500 MG TAB.X 3,PFIZER,945.09
9,8449,TRITAB SUSP.ORAL X 15 ML,SIDUS,961.16
